# Diving into Deep Learning

In this assignment, we will dive right using machine learning to model how bad climate change will be. 

We will use the climate bench dataset, whose processing is covered in the example notebook. 

In [2]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Loading Data into Batches
[Loading Data in Pytorch](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [3]:
from torch.utils.data import DataLoader, Dataset

In [5]:
class MyDataset(torch.Dataset):
    def __init__(self, ...):
        pass

    def __len__(self):
        pass

    def __getitem__(self, index):
        # return (X,y) pair
        pass

SyntaxError: invalid syntax (3765869357.py, line 2)

In [6]:
trainingDataset = 0

testDataset = 0

### Define Model Architecture

Helpful Links:
- [Neural Network Class](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
- [Torch Quickstart](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)


In [8]:
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        pass

## Run Training Loop

Helpful Resources:
- [Optimizer Object](https://pytorch.org/docs/stable/optim.html)
- 


In [ ]:
# set hyperparameters
n_neuron       = 64
num_epochs     = 50
learning_rate  = 0.001
minibatch_size = 64
model_num      = 1
N_layers       = 2 # number of hidden layers

In [10]:
model = MyNeuralNetwork().to(device)
print(model)

MyNeuralNetwork()


### Saving and Evaluating Model